In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
df_FIFA = pd.read_csv('/content/drive/MyDrive/FIFA dataset/male_players (legacy).csv', na_values = '-', low_memory=False)
df_player22 = pd.read_csv('/content/drive/MyDrive/FIFA dataset/players_22-1.csv', na_values = '-', low_memory=False)
df_FIFA.shape



(161583, 110)

In [9]:
df_FIFA

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [10]:
def DropColumns(df_FIFA):
  LessMissing =[]
  MoreMissing = []
  for i in df_FIFA.columns:
    if "url" in i:
        MoreMissing.append(i)
        # use 0.3 for the 30%
    if((df_FIFA[i].isnull().sum())<(0.3*(df_FIFA.shape[0]))):
      LessMissing.append(i)
    else:
      MoreMissing.append(i)
  # df_FIFA = df_FIFA[LessMissing]
  df_FIFA.drop(MoreMissing, axis = 1, inplace = True)
  return df_FIFA


(161583, 100)

In [11]:
def RemoveNANs(df_FIFA2):
  df_FIFA2 = DropColumns(df_FIFA2)
  # Separating features into numeric and categorical variable
  numeric_data = df_FIFA2.select_dtypes(include = np.number)
  non_numeric_data = df_FIFA2.select_dtypes(include = ['object'])
  label_encoders = {}
  # handling missing values in the non numeric data
  for column in non_numeric_data.columns:
    le = LabelEncoder()
    non_numeric_data[column] = le.fit_transform(non_numeric_data[column].astype(str))
    label_encoders[column] = le
    # handling missing values in the non numeric data
  imp = IterativeImputer(max_iter = 10, random_state = 0)
  numeric_data = pd.DataFrame(np.round(imp.fit_transform(numeric_data)), columns = numeric_data.columns)
  Y = numeric_data['overall'].astype(int)
  X = pd.concat([numeric_data, non_numeric_data], axis=1)
  correlations = X.corr()['overall'].sort_values(ascending=False)
  strong_correlations = correlations[:7]
  strong_correlations_columns  = strong_correlations.index.tolist()
  strong_correlations_columns
  strong_correlations_columns.remove('overall')
  X = X[strong_correlations_columns]
  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  return X, Y




player_id                             0
fifa_version                          0
fifa_update                           0
overall                               0
potential                             0
value_eur                          2053
wage_eur                           1761
age                                   0
height_cm                             0
weight_kg                             0
league_id                          1773
league_level                       2717
club_team_id                       1773
club_jersey_number                 1773
club_contract_valid_until_year     1773
nationality_id                        0
weak_foot                             0
skill_moves                           0
international_reputation              0
pace                              17969
shooting                          17969
passing                           17969
dribbling                         17969
defending                         17969
physic                            17969


(161583, 41)


   player_id  fifa_version  fifa_update  overall  potential    value_eur  \
0   158023.0          15.0          2.0     93.0       95.0  100500000.0   
1    20801.0          15.0          2.0     92.0       92.0   79000000.0   
2     9014.0          15.0          2.0     90.0       90.0   54500000.0   
3    41236.0          15.0          2.0     90.0       90.0   52500000.0   
4   167495.0          15.0          2.0     90.0       90.0   63500000.0   

   wage_eur   age  height_cm  weight_kg  ...  mentality_penalties  \
0  550000.0  27.0      169.0       67.0  ...                 76.0   
1  375000.0  29.0      185.0       80.0  ...                 85.0   
2  275000.0  30.0      180.0       80.0  ...                 80.0   
3  275000.0  32.0      195.0       95.0  ...                 91.0   
4  300000.0  28.0      193.0       92.0  ...                 37.0   

   mentality_composure  defending_marking_awareness  \
0                 90.0                         25.0   
1                 

0         93
1         92
2         90
3         90
4         90
          ..
161578    46
161579    46
161580    46
161581    46
161582    46
Name: overall, Length: 161583, dtype: int64

In [15]:
pd.Series(Y).value_counts().head()

overall
64    9826
65    9795
66    9645
67    9583
63    8919
Name: count, dtype: int64

In [22]:
print(X.isnull().sum())
print(X.shape)

player_id       0
fifa_version    0
fifa_update     0
overall         0
potential       0
               ..
lcb             0
cb              0
rcb             0
rb              0
gk              0
Length: 100, dtype: int64
(161583, 100)


In [23]:
# Calculate the correlation between each variable and player's overall rating
correlations[0:60]

overall                       1.000000
movement_reactions            0.845753
mentality_composure           0.699994
potential                     0.695362
wage_eur                      0.611981
rcm                           0.607929
cm                            0.607929
lcm                           0.607929
lf                            0.587947
cf                            0.587947
rf                            0.587947
rw                            0.572444
lw                            0.572444
ram                           0.567905
cam                           0.567905
lam                           0.567905
value_eur                     0.567270
rm                            0.558515
lm                            0.558515
st                            0.552396
ls                            0.552396
rs                            0.552396
ldm                           0.544101
rdm                           0.544101
cdm                           0.544101
lwb                      

In [25]:

X.head()

,movement_reactions,mentality_composure,potential,wage_eur,rcm,cm,lcm
0,94.0,90.0,95.0,550000.0,330,330,330
1,90.0,93.0,92.0,375000.0,315,315,315
2,89.0,92.0,90.0,275000.0,321,321,321
3,85.0,93.0,90.0,275000.0,306,306,306
4,89.0,57.0,90.0,300000.0,72,72,72


In [26]:
# The scaling method it means compressing the variabiliaty of your values


X.shape

(161583, 7)

In [27]:
X, Y = RemoveNANs(df_FIFA)
Xtrain, Xtest, Ytrain,Ytest = train_test_split(X, Y,test_size = 0.2, random_state=42, stratify=Y)
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape

((129266, 7), (32317, 7), (129266,), (32317,))

# Base Training Phase

## 1] Ridge Model

In [28]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Train the model
model = Ridge() # you replace put ElasticNet or lasso on the Ridge function to compare and  train
model.fit(Xtrain, Ytrain)

#Test the model
y_pred = model.predict(Xtest)

print(f""" Mean Absolute Error = {mean_absolute_error(y_pred, Ytest)},
           Mean Squared Error = {mean_squared_error(y_pred, Ytest)},
           Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, Ytest))},
           r2 score = {r2_score(y_pred, Ytest)}
           """)

 Mean Absolute Error = 2.252658637418994,
           Mean Squared Error = 8.514179071900422,
           Root Mean Squared Error = 2.9179066249454286,
           r2 score = 0.7918681518857573
           


## 2] ElasticNet Model

In [29]:
from sklearn.linear_model import ElasticNet
#Train the model
model = ElasticNet() # you replace put ElasticNet or lasso on the Ridge function to compare and  train
model.fit(Xtrain, Ytrain)

#Test the model
y_pred = model.predict(Xtest)

print(f""" Mean Absolute Error = {mean_absolute_error(y_pred, Ytest)},
           Mean Squared Error = {mean_squared_error(y_pred, Ytest)},
           Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, Ytest))},
           r2 score = {r2_score(y_pred, Ytest)}
           """)

 Mean Absolute Error = 2.644746418374924,
           Mean Squared Error = 11.540377694860851,
           Root Mean Squared Error = 3.397113141309964,
           r2 score = 0.508132550035012
           


## 3] Lasso Model

In [30]:
from sklearn.linear_model import Lasso
#Train the model
model = Lasso() # you replace put ElasticNet or lasso on the Ridge function to compare and  train
model.fit(Xtrain, Ytrain)

#Test the model
y_pred = model.predict(Xtest)

print(f""" Mean Absolute Error = {mean_absolute_error(y_pred, Ytest)},
           Mean Squared Error = {mean_squared_error(y_pred, Ytest)},
           Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, Ytest))},
           r2 score = {r2_score(y_pred, Ytest)}
           """)

 Mean Absolute Error = 2.5014801478551787,
           Mean Squared Error = 10.168801180760608,
           Root Mean Squared Error = 3.1888557792350234,
           r2 score = 0.6246863707678154
           


## 4] Linear Regression Model

In [31]:
from sklearn.linear_model import LinearRegression

# Train and evaluate linear regression model
linear_model = LinearRegression()
linear_model.fit(Xtrain, Ytrain)
y_pred_linear = linear_model.predict(Xtest)
print(f""" Mean Absolute Error = {mean_absolute_error(y_pred_linear, Ytest)},
           Mean Squared Error = {mean_squared_error(y_pred_linear, Ytest)},
           Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred_linear, Ytest))},
           r2 score = {r2_score(y_pred_linear, Ytest)}
           """)

 Mean Absolute Error = 2.2526586723776347,
           Mean Squared Error = 8.514180040173825,
           Root Mean Squared Error = 2.9179067908646132,
           r2 score = 0.7918697546192955
           


## 5] Decision Tree Regressor Model

In [32]:
from sklearn.tree import DecisionTreeRegressor

# Train and evaluate  Decision Tree Regressor model
dtree = DecisionTreeRegressor(max_depth = 12) #Change it to 7 and 12 and compare the values
dtree.fit(Xtrain, Ytrain)
y_pred = dtree.predict(Xtest)
print(f""" Mean Absolute Error = {mean_absolute_error(y_pred, Ytest)},
           Mean Squared Error = {mean_squared_error(y_pred, Ytest)},
           Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, Ytest))},
           r2 score = {r2_score(y_pred, Ytest)}
           """)

 Mean Absolute Error = 1.6597229694964204,
           Mean Squared Error = 5.414705388245915,
           Root Mean Squared Error = 2.3269519522856323,
           r2 score = 0.8798771628774491
           


# Training and Evaluating Ensemble Models

In [33]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from xgboost import XGBRegressor

In [34]:
rf_model = RandomForestRegressor(random_state=42)
# Define parameter grids for RandomForestRegressor model
rf_param_grid = {
    'n_estimators': [500, 600],
    'max_depth': [2, 10, 20],
    'min_samples_split': [2, 5]
}
# Perform grid search with cross-validation
rf_grid_search = GridSearchCV(estimator=rf_model, param_grid=rf_param_grid, cv=5, scoring='neg_mean_squared_error')
rf_grid_search.fit(Xtrain, Ytrain)
rf_r2 = r2_score(Ytest, rf_grid_search.predict(Xtest))
rf_best_model = rf_grid_search.best_estimator_
rf_best_model.fit(Xtrain, Ytrain)
rf_y_pred = rf_best_model.predict(Xtest)
rf_mse = mean_squared_error(Ytest, rf_y_pred)
cross_val_score(rf_best_model, Xtrain, Ytrain, cv=5, scoring='neg_mean_squared_error')

print("Random Forest MSE:", rf_mse)
print("Random Forest R2:", rf_r2)
print("Random Forest best parameters:", rf_grid_search.best_params_)
print("Random Forest best score:", rf_grid_search.best_score_)
print("Random Forest best estimator:", rf_grid_search.best_estimator_)
print("Random Forest feature importances:", rf_grid_search.best_estimator_.feature_importances_)


Random Forest MSE: 4.6109147650477205
Random Forest R2: 0.9069695275122313
Random Forest best parameters: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 200}
Random Forest best score: -4.722825269499037
Random Forest best estimator: RandomForestRegressor(max_depth=20, min_samples_split=5, n_estimators=200,
                      random_state=42)
Random Forest feature importances: [0.27009992 0.05536611 0.08778794 0.53905453 0.01599732 0.01581919
 0.015875  ]


In [38]:
xgb_model = XGBRegressor(random_state=42)
xgb_param_grid = {
    'n_estimators': [600, 650],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb_grid = GridSearchCV(xgb_model, xgb_param_grid, cv=5, scoring='neg_mean_squared_error')
xgb_grid.fit(Xtrain, Ytrain)
xgb_grid_search = xgb_grid.best_estimator_
xgb_grid_search.fit(Xtrain, Ytrain)
xgb_y_pred = xgb_grid_search.predict(Xtest)
xgb_mse = mean_squared_error(Ytest, xgb_y_pred)
xgb_cross_val = cross_val_score(xgb_grid_search, Xtrain, Ytrain, cv=5, scoring='neg_mean_squared_error')
xgb_r2 = r2_score(Ytest, xgb_y_pred)

print("XGBoost MSE:", xgb_mse)
print("XGBoost R2:", xgb_r2)
print("XGBoost best score:", xgb_grid.best_score_)
print("XGBoost best estimator:", xgb_grid.best_estimator_)
print("XGBoost feature importances:", xgb_grid.best_estimator_.feature_importances_)
print("XGBoost cross validation score:", xgb_cross_val)


XGBoost MSE: 4.40201654518403
XGBoost R2: 0.9111842877249092
XGBoost best score: -4.464424588597028
XGBoost best estimator: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)
XGBoost feature importances: [0.18875691 0.03650922 0.06344959 0.68400955 0.02727478 0.
 0.        ]
XGBoost cross validation score: [-

In [41]:
gb_model = GradientBoostingRegressor(random_state=42)
gb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}
gb_grid = GridSearchCV(gb_model, gb_param_grid, cv=5, scoring='neg_mean_squared_error')
# gb_grid_search = GridSearchCV(estimator=gb_model, param_grid=gb_param_grid, cv=5, scoring='neg_mean_squared_error')
gb_grid.fit(Xtrain, Ytrain)
gb_grid_search = gb_grid.best_estimator_
gb_grid_search.fit(Xtrain, Ytrain)
gb_y_pred = gb_grid_search.predict(Xtest)
gb_mse = mean_squared_error(Ytest, gb_y_pred)
gb_cross_val = cross_val_score(gb_grid_search, Xtrain, Ytrain, cv=5, scoring='neg_mean_squared_error')
gb_r2 = r2_score(Ytest, gb_y_pred)


print("Gradient Boosting MSE:", gb_mse)
print("Gradient Boosting R2:", gb_r2)
print("Gradient Boosting cross validation score:", gb_cross_val)
#print("Gradient Boosting feature importances:", gb_grid_search.feature_importances_)
print("Gradient Boosting feature importances:", gb_grid_search.feature_importances_) # Changed line
print("Gradient Boosting best parameters:", gb_grid.best_params_)
print("Gradient Boosting best score:", gb_grid.best_score_)
print("Gradient Boosting best estimator:", gb_grid.best_estimator_)



Gradient Boosting MSE: 4.379827914838064
Gradient Boosting R2: 0.9116319686884762
Gradient Boosting cross validation score: [-4.47063597 -4.3569076  -4.50318918 -4.44685793 -4.45732284]
Gradient Boosting feature importances: [0.28084001 0.05249239 0.08342285 0.54632814 0.0104786  0.01334175
 0.01309627]
Gradient Boosting best parameters: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200}
Gradient Boosting best score: -4.446982703892174
Gradient Boosting best estimator: GradientBoostingRegressor(max_depth=10, n_estimators=200, random_state=42)


In [40]:
adb_model = AdaBoostRegressor(random_state=42)
adb_param_grid = {
    'n_estimators': [700, 800],
    'learning_rate': [0.01, 0.1, 0.2]
}
adb_grid_search = GridSearchCV(estimator=adb_model, param_grid=adb_param_grid, cv=5, scoring='neg_mean_squared_error')
adb_grid_search.fit(Xtrain, Ytrain)
adb_best_model = adb_grid_search.best_estimator_
adb_best_model.fit(Xtrain, Ytrain)
adb_y_pred = adb_best_model.predict(Xtest)
adb_mse = mean_squared_error(Ytest, adb_y_pred)
adb_cross_val = cross_val_score(adb_best_model, Xtrain, Ytrain, cv=5, scoring='neg_mean_squared_error')
adb_r2 = r2_score(Ytest, adb_y_pred)
# adb_best_params = adb_grid_search.best_params_
# adb_best_score = adb_grid_search.best_score_
# adb_best_estimator = adb_grid_search.best_estimator_

print("AdaBoosting Best Model:", adb_best_model)
print("AdaBoosting MSE:", adb_mse)
print("AdaBoosting R2:", adb_r2)
print("AdaBoosting cross validation score:", adb_cross_val)
# print("AdaBoosting feature importances:", adb_grid_search.feature_importances_)
print("AdaBoosting feature importances:", adb_best_model.feature_importances_)
print("Gradient Boosting best parameters:", adb_grid_search.best_params_)
print("AdaBoosting best model feature importances:", adb_best_model.feature_importances_)


AdaBoosting Best Model: AdaBoostRegressor(learning_rate=0.1, n_estimators=100, random_state=42)
AdaBoosting MSE: 7.411957954582061
AdaBoosting R2: 0.8504552814983322
AdaBoosting cross validation score: [-7.60317368 -7.35121774 -7.38626191 -7.4621011  -7.37880034]


AttributeError: 'GridSearchCV' object has no attribute 'feature_importances_'

In [53]:
final_model = gb_grid_search

In [58]:
# import joblib
# # joblib
# final_model = gb_grid_search
# # Save the trained model using pickle
# with open('gb_grid_search.pkl', 'wb') as f:
#     pkl.dump(final_model, f)
# # Save the trained model using joblib
# joblib.dump(final_model, 'gb_grid_search.pkl')

# # import pickle


import joblib
import pickle as pkl

# Assuming `gb_grid_search` is your trained model
final_model = gb_grid_search

# Save the trained model using joblib
joblib.dump(final_model, '/content/drive/MyDrive/Colab Notebooks/gb_grid_search_joblib.pkl')







# # Save the trained model using pickle
# with open('gb_grid.pkl', 'wb') as f:
#     pickle.dump(final_model, f)


# # Save the trained model using joblib
# joblib.dump(final_model, 'gb_grid.joblib')




# pkl.dump(model, open(model.__class__.__name__+'pkl', 'wb'))


# # Save the trained model using pickle
# with open('random_forest_model.pkl', 'wb') as f:
#     pickle.dump(xgb_grid, f)
# # Save the trained model using joblib
# joblib.dump(xgb_grid, 'random_forest_model.joblib')



['/content/drive/MyDrive/Colab Notebooks/gb_grid_search_joblib.pkl']